# Python Code Review Agent

This notebook demonstrates the use of the `smolagents` library to create an agent that:
1. Reads a Python file.
2. Parses it to extract function signatures and class definitions.
3. Reviews these functions using a Large Language Model (LLM).
4. Documents the findings in a Markdown file.

Ensure you have the `smolagents` library installed:


In [ ]:
!pip install -q smolagents litellm

In [ ]:
import ast
import os
from smolagents import CodeAgent, HfApiModel, LiteLLMModel, tool

model_id = ("ollama_chat/llama3.2",)

model_id = "ollama/deepseek-coder-v2:latest"
model_id = "ollama/codellama:13b"

In [ ]:
model = LiteLLMModel(model_id=model_id, api_key="ollama")
agent = CodeAgent(tools=[], model=model, add_base_tools=True)

agent.run("Could you give me the 7th Fibonacci number, integer, from recursive calculation?")

In [ ]:
@tool
def read_python_file(file_path: str) -> str:
    """
    Returns the content of a Python file.

    Args:
        file_path: Path to the Python file.
    """
    with open(file_path, "r") as file:
        return file.read()

In [ ]:
@tool
def parse_python_code(code: str) -> list[str]:
    """
    Returns the parsed functions and class definitions from Python code.

    Args:
        code: Python code as a string.

    Returns:
        list: List of function and class signatures.
    """
    tree = ast.parse(code)
    definitions = []
    for node in ast.walk(tree):
        if isinstance(node, ast.FunctionDef):
            definitions.append(f"Function: {node.name}{ast.unparse(node.args)}")
        elif isinstance(node, ast.ClassDef):
            definitions.append(f"Class: {node.name}")
    return definitions

In [ ]:
@tool
def write_findings_to_markdown(findings: list[str], output_path: str) -> str:
    """
    Writes the review findings to a Markdown file.

    Args:
        findings: List of tuples containing code signatures and their reviews.
        output_path: Path to the output Markdown file.
    """
    result = "# Code Review Findings\n\n"

    for signature, review in findings:
        result += f"## {signature}\n"
        result += f"**Review Findings:** {review}\n\n"

    with open(output_path, "w") as md_file:
        md_file.write(result)

    return result

In [ ]:
def review_code_with_llm(agent, code_snippet):
    """
    Uses the LLM agent to review a code snippet.

    Args:
        agent (CodeAgent): The initialized CodeAgent.
        code_snippet (str): The code snippet to review.

    Returns:
        str: Review findings from the LLM.
    """
    prompt = f"Review the following Python code:\n\n{code_snippet}\n\nProvide feedback:"
    return agent.run(prompt)

In [ ]:
import pathlib

# Where am I?
# os.getcwd()
print(pathlib.Path().absolute())

# Note: this is the project root directory!
file_to_review = pathlib.Path("notebooks/review_test_script.py")

assert file_to_review.exists(), "File not found!"

print(str(file_to_review))

results_file_name = file_to_review.with_suffix(".md")
print(str(results_file_name))

In [ ]:
# Ensure you have your Hugging Face API token set in the environment
api_key = os.getenv("HF_TOKEN")

if api_key:
    print("API key found, using Hugging Face API Model.")
    model = HfApiModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct", api_key=api_key)
else:
    print("API key not found, using Local Ollama Model.")
    model_id = "codellama:13b"
    model = LiteLLMModel(model_id=model_id, api_key="ollama")

agent = CodeAgent(model=model, tools=[parse_python_code, read_python_file, write_findings_to_markdown])

In [ ]:
task = f"""
Review the code in the file `{file_to_review}` and provide feedback on the functions and classes defined in the file.

Write the review findings to a Markdown file: `{results_file_name}`.
"""

In [ ]:
result = agent.run(task)
print(result)

In [ ]:
# Cell 8: Main Execution
# Specify the path to the Python file to be reviewed

python_file_path = "path/to/your_script.py"
# Specify the path for the output Markdown file
output_markdown_path = "code_review_findings.md"

# Step 1: Read the Python file
code_content = read_python_file(python_file_path)

# Step 2: Parse the code to extract definitions
definitions = parse_python_code(code_content)

# Step 3: Review each definition using the LLM
findings = []
for definition in definitions:
    review = review_code_with_llm(agent, definition)
    findings.append((definition, review))

# Step 4: Write the findings to a Markdown file
write_findings_to_markdown(findings, output_markdown_path)

print(f"Code review completed. Findings are saved in {output_markdown_path}.")

## Proof of Concept: Example Review Results on a File

In [ ]:
from smolagents import tool


@tool
def review_file(file_path: str, style_guide: str) -> str:
    """Review a file for a certain style guide.

    Args:
        file_path: The path to the file to review.
        style_guide: The style guide to review against.

    Returns:
        str: The review results.
    """
    with open(file_path, "r") as file:
        content = file.read()

    # Placeholder for actual review logic
    review_results = f"Reviewing '{file_path}' against {style_guide}...\n"

    # TODO: run the actual review logic, passing content and style_guide to an LLM
    review_results += "\nNot yet implemented."

    return review_results


example_file_path = "notebooks/review_test_script.py"
example_style_guide = "PEP 8"

review_results = review_file(example_file_path, example_style_guide)
print(review_results)